## Preprocessing
Unlike LDA we should not toy too much with the sentence structure as ABAE uses word embeddings and needs the sequence information to weight the terms based on the surrounding context. One question remains:

**Should we work on sentence level or full reviews? Let's try a first simple comparison**

### Full-reviews

In [1]:
from main.abae.model_manager import ABAEManager
from main.abae.config import ABAEManagerConfig

corpus = "../dataset/output/default/pre_processed.80k.csv"
default_config = ABAEManagerConfig(name="abae_default_ds")
abae_manager = ABAEManager.from_scratch(default_config, corpus)

Pandas Apply:   0%|          | 0/60640 [00:00<?, ?it/s]

INFO:gensim.utils:loading Word2Vec object from output\abae_default_ds\abae_default_ds.embeddings.model
DEBUG:smart_open.smart_open_lib:{'uri': 'output\\abae_default_ds\\abae_default_ds.embeddings.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
INFO:gensim.utils:loading wv recursively from output\abae_default_ds\abae_default_ds.embeddings.model.wv.* with mmap=None
INFO:gensim.utils:setting ignored attribute cum_table to None
INFO:gensim.utils:Word2Vec lifecycle event {'fname': 'output\\abae_default_ds\\abae_default_ds.embeddings.model', 'datetime': '2025-03-13T23:27:07.992300', 'gensim': '4.3.3', 'python': '3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'loaded'}


Loading the existing found model as requested in path ./output/abae_default_ds/abae_default_ds.aspect_embeddings.model


In [2]:
history, _ = abae_manager.train(corpus)
model = abae_manager.get_compiled_model(refresh=False)

Generating numeric representation for each word of ds.


Pandas Apply:   0%|          | 0/60640 [00:00<?, ?it/s]

Max sequence length calculation in progress...
We loose information on 1340(2.2097625329815305% of ds).


KeyboardInterrupt: 

Latest run result:
```
Max Margin loss: [4.6614, 4.6604]
```

In [ ]:
model = abae_manager.get_compiled_model(refresh=False)

In [ ]:
from evaluation import ABAEEvaluationProcessor
import pandas as pd

test_corpus_path = "../dataset/output/default/pre_processed.80k.test.csv"
inv_vocab = abae_manager.generator.emb_model.model.wv.index_to_key
processor = ABAEEvaluationProcessor(abae_manager, pd.read_csv(test_corpus_path))

In [ ]:
from main.abae.dataset import PositiveNegativeABAEDataset
import pandas as pd
from torch.utils.data import DataLoader

test_corpus_path = "../dataset/output/default/pre_processed.80k.test.csv"
df = pd.read_csv(test_corpus_path)

npmi_coh = processor.c_npmi_coherence_model(top_n=10)
npmi_coherence = npmi_coh.get_coherence()

cv_coh = processor.c_v_coherence_model(top_n=100)
cv_coherence = cv_coh.get_coherence()

vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
test_ds = PositiveNegativeABAEDataset(df, vocabulary, max_seq_len, negative_sample_size)

res = model.evaluate(DataLoader(test_ds, batch_size=default_config.batch_size))
print(f"Max margin reconstruction result: {res}")

In [ ]:
processor.silhouette_score()

In [ ]:
print(f"NPMI coherence: {npmi_coherence}")
print(f"CV score: {cv_coherence}")
print(f"Max margin reconstruction result: {res}")

Results of the latest run:
```
NPMI coherence: -0.23037988688672537
CV score: 0.5646751917101251
Max margin reconstruction result: [4.745960235595703, 4.744936943054199]
```

In [ ]:
list(processor.extract_top_k_words(10, 10))

### Sentence-split reviews

In [ ]:
from main.abae.model_manager import ABAEManagerConfig, ABAEManager

corpus = "../dataset/output/default_sentences/pre_processed.80k.csv"
default_config = ABAEManagerConfig(name="abae_sent_ds")
abae_manager = ABAEManager.from_scratch(default_config, corpus, False)

In [ ]:
abae_manager

In [ ]:
history, _ = abae_manager.train(corpus)
model = abae_manager.get_compiled_model(refresh=False)

In [ ]:
# print history
history.history

In [ ]:
model = abae_manager.get_inference_model(refresh=True)

In [ ]:
model.get_layer('emb')

In [ ]:
b.dataset.tolist()

In [ ]:
b[1][0]

In [ ]:
from main.abae.dataset import ABAEDataset, PositiveNegativeABAEDataset
from torch.utils.data import DataLoader
import numpy as np

vocabulary = abae_manager.generator.emb_model.vocabulary()
test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"
b = ABAEDataset(test_corpus_path, vocabulary, 80)
pred = model.predict(DataLoader(b))
pred

In [ ]:
aspect_embeddings.shape

In [ ]:
np.stack(b.dataset.map(lambda x: np.array(x)).to_numpy())

In [ ]:
aspect_embeddings[0][-1]

In [ ]:
w_embs[0][-1]

In [ ]:
n = [w[w != 0] for w in w_embs]

In [ ]:
w_embs[0]

In [ ]:
w_embs[0][w_embs[0] != 0]

In [ ]:
n_em = []
for emb, att in zip(embeddings, attention):
    w = (att[..., np.newaxis] * emb.numpy()).sum(0)
    # w = Weight()([att, emb.cuda() ])
    n_em.append(w)
n_em[0]

In [ ]:
n_em[0]

In [ ]:
from sklearn.metrics import silhouette_score
from main.abae.layer import Weight

mm = abae_manager.get_compiled_model()

attention = pred[0]
embeddings = mm.get_layer(index=1)(np.stack(b.dataset.map(lambda x: np.array(x)).to_numpy())).cpu()

n_em = []
for emb, att in zip(embeddings, attention):
    w = (att[..., np.newaxis] * emb.numpy()).sum(0)
    # w = Weight()([att, emb.cuda() ])
    n_em.append(w)
n_em[0]

# Prediceted aspect

labels = np.argmax(pred[1], axis=1)

silhouette_score(w_embs.cpu(), labels)

In [ ]:
silhouette_score(n_em, labels, metric='cosine')  # -0.06 vs 0.10

In [ ]:
run_result = dict()
# Questa va in caso perdi quella in processor come funzioni
vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
eval_ds = PositiveNegativeABAEDataset(test_corpus_path, vocabulary, max_seq_len, negative_sample_size)

model = abae_manager.get_compiled_model(refresh=False)
run_result['max_margin_loss'] = model.evaluate(DataLoader(eval_ds, batch_size=default_config.batch_size))
ev_model = abae_manager.get_inference_model()

pred_val_data = ABAEDataset(test_corpus_path, vocabulary, max_seq_len)
att, labels = ev_model.predict(DataLoader(pred_val_data, batch_size=default_config.batch_size, shuffle=False))

# todo verifica
# Calculate the embeddings of the model.
embeddings = model.get_layer(index=1)(np.stack(pred_val_data.dataset.map(lambda x: np.array(x))))
# Weight the embeddings to create sentence embeddings
w_embs = [(att[..., np.newaxis] * emb.cpu().numpy()).sum(0) for emb, att in zip(embeddings, att)]
run_result['silhouette_score'] = silhouette_score(w_embs, np.argmax(labels, axis=1))

In [ ]:
run_result

In [ ]:
from evaluation import ABAEEvaluationProcessor

inv_vocab = abae_manager.generator.emb_model.model.wv.index_to_key
processor = ABAEEvaluationProcessor.generate_for_model(model, inv_vocab)

In [ ]:
from main.abae.dataset import PositiveNegativeABAEDataset
import pandas as pd
from torch.utils.data import DataLoader

test_corpus_path = "../dataset/output/default_sentences/pre_processed.80k.test.csv"
df = pd.read_csv(test_corpus_path)

npmi_coh = processor.c_npmi_coherence_model(top_n=10, ds=df['comments'].apply(lambda x: x.split(' ')))
npmi_coherence = npmi_coh.get_coherence()

cv_coh = processor.c_v_coherence_model(top_n=100, ds=df['comments'].apply(lambda x: x.split(' ')))
cv_coherence = cv_coh.get_coherence()

vocabulary = abae_manager.generator.emb_model.vocabulary()
max_seq_len = default_config.max_seq_len
negative_sample_size = default_config.negative_sample_size
test_ds = PositiveNegativeABAEDataset(df, vocabulary, max_seq_len, negative_sample_size)

res = model.evaluate(DataLoader(test_ds, batch_size=default_config.batch_size))
print(f"Max margin reconstruction result: {res}")

In [ ]:
print(f"NPMI coherence: {npmi_coherence}")
print(f"CV score: {cv_coherence}")
print(f"Max margin reconstruction result: {res}")

Results for the run:
```
NPMI coherence: -0.3117940799781337
CV score: 0.6031670887985386
Max margin reconstruction result: [5.261548042297363, 5.262136936187744]
```

In [ ]:
list(processor.extract_top_k_words(11, 10))

I know that doing a  comparison on a single run is not that meaningful. <br>
I could do k-CV to estimate the expected model loss to get a valid analysis. <br>
But for the sake of the experiment we consider this good enough.

For the future work and as one by the proposed ABAE paper we won't be splitting up reviews in sentences but use the full review as the model does not increase much if not done like this.